In [1]:
! pip install -q pytorch-lightning
! pip install -q transformers

     |████████████████████████████████| 708 kB 18.9 MB/s 
     |████████████████████████████████| 529 kB 64.3 MB/s 
     |████████████████████████████████| 5.3 MB 25.5 MB/s 
     |████████████████████████████████| 163 kB 50.4 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 


In [2]:
# Huggingface transformers
import torch
import numpy as np
import pandas as pd
import transformers
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertModel,BertTokenizer,AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# from sklearn.model_selection import train_valid_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [3]:
df_train = pd.read_csv("train_preprocess.csv")
df_valid = pd.read_csv("valid_preprocess.csv")

In [4]:
df_train.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,kebersihan kurang...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut
1,"sangat mengecewakan... hotel bad image, kebers...",neut,neut,neut,neut,neg,neut,neut,neut,neut,neut
2,Tempat nyaman bersih tapi tv terlalu tinggi ti...,neut,neut,neut,neut,pos,neut,neut,neut,neg,neut
3,"semuanya bagus sesuai profile,dan harga promo ...",neut,neg,neut,pos,neut,neut,neut,neut,neut,neut
4,"Tempat tidur sangat keras, bantal besar dan ke...",neg,neg,neut,neut,neut,neg,neut,neut,neut,neut


In [5]:
df_train.describe()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
count,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283
unique,2283,4,3,3,3,4,4,4,3,3,3
top,kebersihan kurang...,neut,neut,neut,neut,neut,neut,neut,neut,neut,neut
freq,1,1814,1922,1911,2023,1350,1613,1649,2108,2075,1928


In [6]:
df_valid.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [7]:
df = pd.concat([df_train, df_valid])

In [8]:
def refine_label(lb):
  return ' '.join([aspect[i]+"-"+lb[i] for i in range(len(aspect))])

In [9]:
def assign_label_list(dft):
  temp = [dft[i] for i in aspect]
  label_list.append(temp)

In [10]:
aspect = ['ac', 'air_panas', 'bau', 'general', 'kebersihan', 'linen', 'service', 'sunrise_meal', 'tv', 'wifi']
labels = {'neg': 0, 'neut': 1, 'neg_pos': 2, 'pos': 3
          }
label_list = []
tqdm.pandas()
df.progress_apply(assign_label_list, axis=1)
df['label'] = label_list
df.label = df.label.apply(refine_label)
df = df.drop([i for i in aspect], axis=1)
df.head()

100%|██████████| 2568/2568 [00:00<00:00, 27405.59it/s]


,review,label
0,kebersihan kurang...,ac-neut air_panas-neut bau-neut general-neut k...
1,"sangat mengecewakan... hotel bad image, kebers...",ac-neut air_panas-neut bau-neut general-neut k...
2,Tempat nyaman bersih tapi tv terlalu tinggi ti...,ac-neut air_panas-neut bau-neut general-neut k...
3,"semuanya bagus sesuai profile,dan harga promo ...",ac-neut air_panas-neg bau-neut general-pos keb...
4,"Tempat tidur sangat keras, bantal besar dan ke...",ac-neg air_panas-neg bau-neut general-neut keb...


In [11]:
df.label = df.label.apply(lambda x: x.split())
mlb = MultiLabelBinarizer()
yt = mlb.fit_transform(df["label"])
print(yt[0])
print(mlb.inverse_transform(yt[0].reshape(1,-1)))
print(mlb.classes_)

[0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0]
[('ac-neut', 'air_panas-neut', 'bau-neut', 'general-neut', 'kebersihan-neg', 'linen-neut', 'service-neut', 'sunrise_meal-neut', 'tv-neut', 'wifi-neut')]
['ac-neg' 'ac-neg_pos' 'ac-neut' 'ac-pos' 'air_panas-neg' 'air_panas-neut'
 'air_panas-pos' 'bau-neg' 'bau-neut' 'bau-pos' 'general-neg'
 'general-neut' 'general-pos' 'kebersihan-neg' 'kebersihan-neg_pos'
 'kebersihan-neut' 'kebersihan-pos' 'linen-neg' 'linen-neg_pos'
 'linen-neut' 'linen-pos' 'service-neg' 'service-neg_pos' 'service-neut'
 'service-pos' 'sunrise_meal-neg' 'sunrise_meal-neut' 'sunrise_meal-pos'
 'tv-neg' 'tv-neut' 'tv-pos' 'wifi-neg' 'wifi-neut' 'wifi-pos']


In [12]:
yt_str = []
for i in yt:
  temp = []
  for j in i:
    temp.append(str(j))
  yt_str.append(' '.join(temp))

In [13]:
def labelled(label):
  tmp = label.split()
  res = []
  for i in tmp:
    res.append(int(i))
  return res

In [14]:
df["labeled"] = yt_str

In [15]:
df.head()

,review,label,labeled
0,kebersihan kurang...,"[ac-neut, air_panas-neut, bau-neut, general-ne...",0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 ...
1,"sangat mengecewakan... hotel bad image, kebers...","[ac-neut, air_panas-neut, bau-neut, general-ne...",0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 ...
2,Tempat nyaman bersih tapi tv terlalu tinggi ti...,"[ac-neut, air_panas-neut, bau-neut, general-ne...",0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 ...
3,"semuanya bagus sesuai profile,dan harga promo ...","[ac-neut, air_panas-neg, bau-neut, general-pos...",0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 ...
4,"Tempat tidur sangat keras, bantal besar dan ke...","[ac-neg, air_panas-neg, bau-neut, general-neut...",1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 ...


In [16]:
df.labeled = df.labeled.apply(labelled)

In [17]:
df.head()

,review,label,labeled
0,kebersihan kurang...,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
1,"sangat mengecewakan... hotel bad image, kebers...","[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
2,Tempat nyaman bersih tapi tv terlalu tinggi ti...,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,"semuanya bagus sesuai profile,dan harga promo ...","[ac-neut, air_panas-neg, bau-neut, general-pos...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
4,"Tempat tidur sangat keras, bantal besar dan ke...","[ac-neg, air_panas-neg, bau-neut, general-neut...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."


In [18]:
df_train = df.iloc[0:2283,:]
df_valid = df.iloc[2283:,:]


In [19]:
df_train.head()

,review,label,labeled
0,kebersihan kurang...,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
1,"sangat mengecewakan... hotel bad image, kebers...","[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
2,Tempat nyaman bersih tapi tv terlalu tinggi ti...,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,"semuanya bagus sesuai profile,dan harga promo ...","[ac-neut, air_panas-neg, bau-neut, general-pos...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
4,"Tempat tidur sangat keras, bantal besar dan ke...","[ac-neg, air_panas-neg, bau-neut, general-neut...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."


In [20]:
df_valid.head()

,review,label,labeled
0,over all baik tapi kalau bisa kolong tempat ti...,"[ac-neut, air_panas-neut, bau-neut, general-po...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, ..."
1,"Ac harus dicek secara Berkala, tanpa harus men...","[ac-neg, air_panas-neut, bau-neut, general-neu...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
2,Nyaman tp ac bocor,"[ac-neg, air_panas-neut, bau-neut, general-neu...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
4,agak kotor di kamar yg lantai 2.,"[ac-neut, air_panas-neut, bau-neut, general-ne...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."


In [21]:
x_train, x_valid = df_train.iloc[:,0].to_numpy(), df_valid.iloc[:,0].to_numpy()
y_train, y_valid = df_train.iloc[:,2].to_numpy(), df_valid.iloc[:,2].to_numpy()

In [22]:
class HotelDataset (Dataset):
    def __init__(self, review, label, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = review
        self.labels = label
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        #token_type_ids = inputs["token_type_ids"]
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.labels[item_idx], dtype=torch.float)
            
        }

In [23]:
class Module (pl.LightningDataModule):
    
  def __init__(self,x_tr,y_tr,x_valid,y_valid,tokenizer,batch_size=16,max_token_len=512):
        super().__init__()
        self.tr_text = x_tr
        self.tr_label = y_tr
        self.valid_text = x_valid
        self.valid_label = y_valid
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

  def setup(self,stage = None):
        self.train_dataset = HotelDataset(review=self.tr_text, label=self.tr_label, tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.valid_dataset  = HotelDataset(review=self.valid_text, label=self.valid_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        
        
  def train_dataloader(self):
        return DataLoader (self.train_dataset,batch_size = self.batch_size, shuffle = True , num_workers=2)

  def val_dataloader(self):
        return DataLoader (self.valid_dataset, batch_size= self.batch_size)

In [24]:
class Classifier(pl.LightningModule):
    # Set up the classifier
  def __init__(self, n_classes=34, steps_per_epoch=None, n_epochs=2, lr=2e-5 ):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size,n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
        
  def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        output = self.classifier(output.pooler_output)
                
        return output
    
    
  def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


  def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
  def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr)
        warmup_steps = self.steps_per_epoch//3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,total_steps)

        return [optimizer], [scheduler]

In [74]:
BERT_MODEL_NAME = "indobenchmark/indobert-base-p2"
Bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
N_EPOCHS = 5
BATCH_SIZE = 2
MAX_LEN = 512
LR = 2e-05
module = Module(x_train,y_train,x_valid,y_valid,
Bert_tokenizer,BATCH_SIZE,MAX_LEN)
module.setup()

In [75]:
# Instantiate the classifier model
steps_per_epoch = len(x_train)//BATCH_SIZE
model = Classifier(n_classes=34, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

In [76]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',# monitored quantity
    filename='HotelReview-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3, #  save the top 3 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [77]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [78]:
# Instantiate the Model Trainer
trainer = pl.Trainer(max_epochs = N_EPOCHS ,gpus = 1,  callbacks=[checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [79]:
trainer.fit(model, module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 124 M 
1 | classifier | Linear            | 26.1 K
2 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
497.870   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [80]:
flat_pred_outs = 0
flat_true_labels = 0

In [81]:
y_val = []
for i in range(len(y_valid)):
  tmp = []
  for j in range(len(y_valid[0])):
    tmp.append(int(y_valid[i][j]))
  y_val.append(np.array(tmp))
y_valid = np.array(y_val)


In [82]:
from torch.utils.data import TensorDataset

# Tokenize all questions in x_valid
input_ids = []
attention_masks = []


for quest in x_valid:
    encoded_quest =  Bert_tokenizer.encode_plus(
                    quest,
                    None,
                    add_special_tokens=True,
                    max_length= MAX_LEN,
                    padding = 'max_length',
                    return_token_type_ids= False,
                    return_attention_mask= True,
                    truncation=True,
                    return_tensors = 'pt'      
    )
    
    # Add the input_ids from encoded question to the list.    
    input_ids.append(encoded_quest['input_ids'])
    # Add its attention mask 
    attention_masks.append(encoded_quest['attention_mask'])
    
# Now convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
y_valid = y_valid.astype(np.int8)
labels = torch.from_numpy(y_valid)

# Set the batch size.  
VALID_BATCH_SIZE = 64 

# Create the DataLoader.
pred_data = TensorDataset(input_ids, attention_masks, labels)
pred_sampler = SequentialSampler(pred_data)
pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=VALID_BATCH_SIZE)

In [83]:
# Put model in evaluation mode
model = model.to(device) # moving model to cuda
model.eval()

# Tracking variables 
pred_outs, true_labels = [], []
#i=0
# Predict 
for batch in pred_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_attn_mask, b_labels = batch
 
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        pred_out = model(b_input_ids,b_attn_mask)
        pred_out = torch.sigmoid(pred_out)
        # Move predicted output and labels to CPU
        pred_out = pred_out.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        #i+=1
        # Store predictions and true labels
        #print(i)
        #print(outputs)
        #print(logits)
        #print(label_ids)
    pred_outs.append(pred_out)
    true_labels.append(label_ids)

In [84]:
# Combine the results across all batches. 
flat_pred_outs = np.concatenate(pred_outs, axis=0)

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

In [85]:
#define candidate threshold values
threshold  = np.arange(0.2,0.51,0.1)
threshold

array([0.2, 0.3, 0.4, 0.5])

In [86]:
# convert probabilities into 0 or 1 based on a threshold value
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

In [87]:
from sklearn import metrics
scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = flat_true_labels.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(flat_pred_outs,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(metrics.f1_score(y_true,y_pred))

In [88]:
# find the optimal threshold
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.4000000000000001


In [89]:
#predictions for optimal threshold
y_pred_labels = classify(flat_pred_outs,opt_thresh)
y_pred = np.array(y_pred_labels).ravel() # Flatten

In [90]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      6840
           1       0.80      0.80      0.80      2850

    accuracy                           0.88      9690
   macro avg       0.86      0.86      0.86      9690
weighted avg       0.88      0.88      0.88      9690



In [91]:
y_pred = mlb.inverse_transform(np.array(y_pred_labels))
y_act = mlb.inverse_transform(flat_true_labels)

df_result = pd.DataFrame({'Review':x_valid,'Actual Label':y_act,'Predicted Label':y_pred})
df_result.head()

,Review,Actual Label,Predicted Label
0,over all baik tapi kalau bisa kolong tempat ti...,"(ac-neut, air_panas-neut, bau-neut, general-po...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
1,"Ac harus dicek secara Berkala, tanpa harus men...","(ac-neg, air_panas-neut, bau-neut, general-neu...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
2,Nyaman tp ac bocor,"(ac-neg, air_panas-neut, bau-neut, general-neu...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,"(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
4,agak kotor di kamar yg lantai 2.,"(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."


In [92]:
df_result.sample(10)

,Review,Actual Label,Predicted Label
92,"Kurang ramah resepsionistnya,tvnya juga gk ber...","(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
163,"Tempatnya bersih, minus wifi nya ga bisa","(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
86,"Air asin, kamar kotor dan wc kotor.","(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
146,Kamar masih ada bau debu. Kamar mandi showerny...,"(ac-neut, air_panas-neg, bau-neg, general-neut...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
129,udah enak to kurang bersih,"(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
263,"Lumayan bagus, hanya air showernya awal panas ...","(ac-neut, air_panas-neg, bau-neut, general-neu...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
33,tidak sesuai. baru kali ini saya menginap di a...,"(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
207,"gak ramah, pelayanan bad service","(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
264,"Menyenangkan, namun wifi yang bermasalah sanga...","(ac-neut, air_panas-neut, bau-neut, general-ne...","(ac-neut, air_panas-neut, bau-neut, general-ne..."
81,Penginapan ini cukup rensentatif dan sayangnya...,"(ac-neut, air_panas-neg, bau-neut, general-neu...","(ac-neut, air_panas-neut, bau-neut, general-ne..."


In [93]:
df_result['Predicted Label'].unique()

array([('ac-neut', 'air_panas-neut', 'bau-neut', 'general-neut', 'kebersihan-neut', 'linen-neut', 'service-neut', 'sunrise_meal-neut', 'tv-neut', 'wifi-neut')],
      dtype=object)

In [94]:
model_path = checkpoint_callback.best_model_path
model_path

'/content/lightning_logs/version_2/checkpoints/HotelReview-epoch=04-val_loss=0.37.ckpt'

In [95]:
Hotelmodel = Classifier.load_from_checkpoint(model_path)
Hotelmodel.eval()

Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [96]:
def predict(question):
    text_enc = Bert_tokenizer.encode_plus(
            question,
            None,
            add_special_tokens=True,
            max_length= MAX_LEN,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True,
            truncation=True,
            return_tensors = 'pt'      
    )
    outputs = Hotelmodel(text_enc['input_ids'], text_enc['attention_mask'])
    pred_out = outputs[0].detach().numpy()
    #print(f'Outputs = {outputs}')
    #print(f'Type = {type(outputs)}')
    #print(f'Pred Outputs = {pred_out}')
    #print(f'Type = {type(pred_out)}')
    #preds = np.round(pred_out)
    preds = [(pred > opt_thresh) for pred in pred_out ]
    #pred_list = [ round(pred) for pred in pred_logits ]
    preds = np.asarray(preds)
    #print(f'Predictions = {preds}')
    #print(f'Type = {type(preds)}')
    #print(mlb.classes_)
    new_preds = preds.reshape(1,-1).astype(int)
    #print(new_preds)
    pred_tags = mlb.inverse_transform(new_preds)
    #print(mlb.inverse_transform(np.array(new_preds)))
    return pred_tags 

In [100]:
import time
start_time = time.time()
sentence = "ac dingin tetapi ada bau tidak enak dan wifi lemot"
res = predict(sentence)
print(res)
print("--- %s seconds ---" % (time.time() - start_time))

[('ac-neut', 'air_panas-neut', 'bau-neut', 'general-neut', 'linen-neut', 'service-neut', 'sunrise_meal-neut', 'tv-neut', 'wifi-neut')]
--- 1.3615741729736328 seconds ---
